In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai
fastai.__version__

In [ ]:
from fastai import *
from fastai_audio import *

from fastai.vision import models

In [ ]:
DATA = Path('data')
NSYNTH = DATA/'audio'

In [ ]:
n_fft = 1024
n_hop = 256
n_mels = 128
sr = 16000
top_db = 60.0

bs = 128

In [ ]:
mel_spec_tfm = Compose([
    Spectrogram(n_fft=n_fft, n_hop=n_hop),
    FrequencyToMel(n_mels=n_mels, n_fft=n_fft, sr=sr, f_min=0., f_max=None),
    SpectrumToDb(ref='max', top_db=top_db, normalized=True)
])

def inputs_tfm(inputs):
    x, y = inputs
    # 1d time domain signal to 2d log melspectrogram (frequency domain)
    x = mel_spec_tfm(x)
    # add channel dimension
    x.unsqueeze_(1)
    return x, y

tfms = [inputs_tfm]

In [ ]:
instrument_family_pattern = r'(\w+)_\w+_\d+-\d+-\d+.wav$'

data = (AudioItemList
            .from_folder(NSYNTH)
            .filter_by_func(lambda fn: 'acoustic' in fn.name)
            .split_by_folder()
            .label_from_re(instrument_family_pattern)
            .databunch(bs=128, tfms=tfms, equal_lengths=True))
data

In [ ]:
data.c, data.classes